# Convolutional Neural Network

### Importing the libraries

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing

In [5]:
!wget "http://dl.dropboxusercontent.com/s/w9aqbqxmj4i2my8/dataset.zip"
!unzip dataset.zip
!ls

URL transformed to HTTPS due to an HSTS policy
--2023-12-12 07:57:10--  https://dl.dropboxusercontent.com/s/w9aqbqxmj4i2my8/dataset.zip
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.80.15, 2620:100:6035:15::a27d:550f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.80.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228733995 (218M) [application/zip]
Saving to: ‘dataset.zip.1’

dataset.zip.1       100%[===================>] 218.14M  16.3MB/s    in 12s     

2023-12-12 07:57:23 (18.0 MB/s) - ‘dataset.zip.1’ saved [228733995/228733995]

Archive:  dataset.zip
replace dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: dataset  dataset.zip  dataset.zip.1  sample_data


### Preprocessing the Training set

In [6]:
# image augmentation: to ensure cnn does not overfit or overlearn
train_datagen = ImageDataGenerator(
    # rescale feature scales values 0 to 255
    rescale=1./255,
    #image augmentation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [7]:
# we don't have to augment the test set; just feature scale them
# more theory after more research
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set', target_size=(64, 64), batch_size=32, class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [17]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [18]:
# kernel size = 3 * 3 matrix so, 3,
# input shape : size of each image is 64,64. colored image: 64,64,3, bw: 64,64,1
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [19]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides=2 ))

### Adding a second convolutional layer

In [20]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size= 3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [21]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [22]:

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [23]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [24]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [25]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 77s 301ms/step - loss: 0.6675 - accuracy: 0.5946 - val_loss: 0.6188 - val_accuracy: 0.6715
Epoch 2/25
250/250 [==============================] - 64s 257ms/step - loss: 0.6085 - accuracy: 0.6720 - val_loss: 0.5755 - val_accuracy: 0.7045
Epoch 3/25
250/250 [==============================] - 63s 250ms/step - loss: 0.5612 - accuracy: 0.7061 - val_loss: 0.5241 - val_accuracy: 0.7585
Epoch 4/25
250/250 [==============================] - 65s 261ms/step - loss: 0.5196 - accuracy: 0.7420 - val_loss: 0.5235 - val_accuracy: 0.7510
Epoch 5/25
250/250 [==============================] - 62s 248ms/step - loss: 0.4913 - accuracy: 0.7651 - val_loss: 0.4976 - val_accuracy: 0.7675
Epoch 6/25
250/250 [==============================] - 64s 254ms/step - loss: 0.4713 - accuracy: 0.7741 - val_loss: 0.4897 - val_accuracy: 0.7615
Epoch 7/25
250/250 [==============================] - 63s 251ms/step - loss: 0.4530 - accuracy: 0.7855 - val_loss: 0.4585 - val_ac

## Part 4 - Making a single prediction

In [29]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64, 64))
# pil format to np.array format
test_image = image.img_to_array(test_image)
# was trained with batch of images (32), single image needs to be in batch size 32
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 52ms/step


In [30]:
print(prediction)

cat


In [31]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
# pil format to np.array format
test_image = image.img_to_array(test_image)
# was trained with batch of images (32), single image needs to be in batch size 32
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 38ms/step


In [32]:
print(prediction)

dog
